In [8]:
import collections
from lung import volume_from_pressure, pressure_from_volume

start_time = 0
end_time = 10000
time_resolution = 250
respiratory_rate = 10
Pi = 15
PEEP = 5
IE = 0.5
"""
							In men 	In women
				Vital capacity 	4.8 	3.1 	IRV + TV + ERV
		Inspiratory capacity 	3.8 	2.4 	IRV + TV
Functional residual capacity 	2.4 	1.8 	ERV + RV
		Total lung capacity 	6.0 	4.2 	IRV + TV + ERV + RV 
"""
Patient_log = collections.namedtuple('Patient_log', ['time', 'pressure_mouth', 'pressure_alveolus', 'pressure_intrapleural', 'lung_volume'])

class Patient:
    def __init__(self, height = 175, weight = 70, sex = 'M', pressure_mouth = 0):
        self.time = 0 # miliseconds
        self.height = height
        self.weight = weight
        self.sex = sex
        self.TLC = 6000 if sex == 'M' else 4200 # todo calculate on age, height weight
        self.pressure_mouth = pressure_mouth
        self.pressure_alveolus = pressure_mouth # start at equlibrium
        v_percent = volume_from_pressure(self.pressure_alveolus, 'Total') #assuming no resp effort
        self.lung_volume = self.TLC * v_percent / 100
        self.pressure_intrapleural = pressure_from_volume(v_percent, 'Chest')
        self.log = []
    
    def status(self):
        return Patient_log(self.time, self.pressure_mouth, self.pressure_alveolus, self.pressure_intrapleural, self.lung_volume)
            
    def advance(self, advance_time = 200, pressure_mouth = 0):
        self.time = self.time + advance_time # miliseconds
        
        status = self.status()
        self.log.append(status)
        return status
        
        
Ventilator_log = collections.namedtuple('Ventilator_log', ['time', 'phase', 'pressure', 'pressure_mouth'])
class Ventilator:
    def __init__(self, mode = "PCV", Pi = 15, PEEP = 5, rate = 10, IE=0.5):
        self.pressure = 0
        self.pressure_mouth = 0
        self.mode = mode
        self.Pi = Pi
        self.mode = mode
        self.PEEP = PEEP
        self.rate = rate
        self.IE = IE
        self.phase = "E"
        self.log = []
        self.time = 0 # miliseconds
    
    def target_pressure(self):
        return self.PEEP if self.phase == "E" else self.Pi
    
    def status(self):
        return Ventilator_log(self.time, self.phase, self.pressure, self.pressure_mouth)
    
    def advance(self, advance_time = 200, pressure_mouth = 0):
        self.time = self.time + advance_time # miliseconds
        self.pressure_mouth = pressure_mouth # cmH2O
        # set phase
        breath_length = 60000 / self.rate # milliseconds
        time_since_inspiration_began = self.time % breath_length
        inspiration_length = breath_length * self.IE / (self.IE + 1)
        new_phase = "I" if time_since_inspiration_began < inspiration_length else "E"
        if new_phase != self.phase:
            self.phase = new_phase
            self.pressure = self.target_pressure()
        status = self.status()
        self.log.append(status)
        return status
    
p = Patient()
v = Ventilator()
print('starting', p.status())
print('vent', v.status())
for current_time in range(start_time, end_time, time_resolution): 
    ventilator_status = v.advance(advance_time = time_resolution)
    print(ventilator_status)
    patient_status = p.advance(advance_time = time_resolution, pressure_mouth = ventilator_status.pressure_mouth)
    print(patient_status)
    


starting Patient_log(time=0, pressure_mouth=0, pressure_alveolus=0, pressure_intrapleural=array(-3.21061621), lung_volume=2510.3091325915266)
vent Ventilator_log(time=0, phase='E', pressure=0, pressure_mouth=0)
Ventilator_log(time=250, phase='I', pressure=15, pressure_mouth=0)
Patient_log(time=250, pressure_mouth=0, pressure_alveolus=0, pressure_intrapleural=array(-3.21061621), lung_volume=2510.3091325915266)
Ventilator_log(time=500, phase='I', pressure=15, pressure_mouth=0)
Patient_log(time=500, pressure_mouth=0, pressure_alveolus=0, pressure_intrapleural=array(-3.21061621), lung_volume=2510.3091325915266)
Ventilator_log(time=750, phase='I', pressure=15, pressure_mouth=0)
Patient_log(time=750, pressure_mouth=0, pressure_alveolus=0, pressure_intrapleural=array(-3.21061621), lung_volume=2510.3091325915266)
Ventilator_log(time=1000, phase='I', pressure=15, pressure_mouth=0)
Patient_log(time=1000, pressure_mouth=0, pressure_alveolus=0, pressure_intrapleural=array(-3.21061621), lung_volume